# Evaluate the SH Coefficients Metrics on Generated Dataset

In [1]:
import os
os.chdir(f'{os.getcwd()}/../../')
WORKING_DIR = './evaluation/trigger'

In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import utils3d as u3d

from datasets.xihe.loader import XiheTestDataset

In [4]:
sv_pairs = np.load(f'{WORKING_DIR}/data/sv_pairs.npy')
nsv_pairs = np.load(f'{WORKING_DIR}/data/nsv_pairs.npy')
pairs = np.concatenate((sv_pairs, nsv_pairs), axis=0)

In [7]:
test_dataset = XiheTestDataset(n_points=1280)

In [8]:
shc_rmse = np.zeros((len(pairs)), dtype=np.float32)

In [9]:
for i in tqdm(range(len(pairs))):
    p = pairs[i]
    m, n = p
    
    shc_m = test_dataset[m][1].numpy()
    shc_n = test_dataset[n][1].numpy()
    
    shc_rmse[i] = np.sqrt(np.mean((shc_m - shc_n) ** 2))

100%|██████████| 1754/1754 [00:01<00:00, 1541.70it/s]


In [10]:
np.save(f'{WORKING_DIR}/data/shc_rmse', shc_rmse)

In [11]:
from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity

In [12]:
psnr = np.zeros((len(pairs)), dtype=np.float32)
ssim = np.zeros((len(pairs)), dtype=np.float32)

In [13]:
for i in tqdm(range(len(pairs))):

    p = pairs[i]
    m, n = p

    sh_m = u3d.SphericalHarmonics.from_array(test_dataset[m][1].numpy())
    sh_n = u3d.SphericalHarmonics.from_array(test_dataset[n][1].numpy())

    c_m = sh_m.reconstruct_to_canvas()
    c_n = sh_n.reconstruct_to_canvas()
    
    data_m = np.clip(c_m.data, 0, 1)
    data_n = np.clip(c_n.data, 0, 1)
    
    psnr[i] = peak_signal_noise_ratio(data_m, data_n)
    ssim[i] = structural_similarity(data_m, data_n, multichannel=True)

100%|██████████| 1754/1754 [01:39<00:00, 17.71it/s]


In [14]:
np.save(f'{WORKING_DIR}/data/shr_psnr', psnr)
np.save(f'{WORKING_DIR}/data/shr_ssim', ssim)